# Data Videos Preprocessing

This notebook has code with the purpose of pre-processing videos for further analysis in ImageJ/FIJI. It can do three things: i) threshold images in a video (make sure that only pure blacks or pure whites are in the video); ii) invert the colors of images in a video; and iii) convert videos into gif's.

In [11]:
import sys
print(sys.executable)

/Users/xavier/opt/anaconda3/bin/python


In [8]:
# Print the paths where Python looks for modules
# print(sys.path)

# Search for the path where moviepy is installed
for path in sys.path:
    if 'moviepy' in path:
        print(f"moviepy is installed at: {path}")

In [4]:
import os
import cv2
import time
import imageio
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image, display, clear_output

## Testing to Process Video

In [10]:
# Output directory to save processed frames
output_directory = 'aggregationA_processed_frames_2'
os.makedirs(output_directory, exist_ok=True)

# Open the video file
video_file = 'aggregationA.mp4'
cap = cv2.VideoCapture(video_file)

if not cap.isOpened():
    print("Error opening video file")

# Print number of frames in video.
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(total_frames)

8897


In [12]:
# Initialize a frame counter
frame_count = 0
start_frame = 0 #450
end_frame = 10000 #6000
threshold_value = 128
square_size = 1360  # Size of the square region to extract
circle_radius = 680

while True and frame_count < end_frame:
    # Read a frame from the video
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1
    
    if frame_count < start_frame:
        continue
    
#     # Display the original image for reference.
#     plt.figure(figsize=(8, 8))
#     plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
#     plt.axis('off')
#     plt.show()
#     clear_output(wait=True)
#     time.sleep(0.25)
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    inverted = 255 - gray
    _, thresholded = cv2.threshold(inverted, threshold_value, 255, cv2.THRESH_BINARY)
    
    # Determine center of the frame
    center_x = frame.shape[1] // 2 + 150
    center_y = frame.shape[0] // 2 + 15
    
    # Place a circle in the center of the frame (for purposes of validation).
    processed = inverted.copy()
#     processed = cv2.circle(thresholded, (center_x,center_y), 10, (255, 255, 255), -1)  # Red centroid
    
#     # Create a mask for the circular region
#     mask = np.zeros_like(processed)
#     cv2.circle(mask, (center_x, center_y), circle_radius, (255, 255, 255), -1)
    
#     # Apply the mask to the thresholded frame
#     processed = cv2.bitwise_and(processed, processed, mask=mask)
    
#     # Calculate the coordinates for the square region
#     top_left_x = center_x - square_size // 2
#     top_left_y = center_y - square_size // 2
#     bottom_right_x = center_x + square_size // 2
#     bottom_right_y = center_y + square_size // 2
    
#     # Trim to square region to finish processing
#     processed = processed[top_left_y:bottom_right_y, top_left_x:bottom_right_x]
    
#     # Display the modified image for reference.
#     plt.figure(figsize=(8, 8))
#     plt.imshow(cv2.cvtColor(processed, cv2.COLOR_BGR2RGB))
#     plt.axis('off')
#     plt.show()
#     time.sleep(0.1)

#     break
    
#     # Clear the output
#     clear_output(wait=True)
    
    # Save processed frame as image
    cv2.imwrite(f'{output_directory}/frame_{frame_count:04d}.jpg', processed)
    
# # Display the modified image for reference.
# plt.figure(figsize=(8, 8))
# plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
# plt.axis('off')
# plt.show()

# cap.release()
# print(frame_count)

In [95]:
# Create GIF from processed frames
images = []
for filename in sorted(os.listdir(output_directory)):
    if filename.endswith('.jpg'):
        images.append(imageio.imread(os.path.join(output_directory, filename)))

d = round(frame_count/60,2)
imageio.mimsave('aggregationC.gif', images, duration=d)  # Adjust duration as needed

OSError: [Errno 28] No space left on device

## Code to Actually Process Video

In [1]:
# Define function to streamline processing of frames in a video.
def process_frame(frame, threshold, square_size, circle_radius, x_offset = 0, y_offset = 0):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    inverted_frame = cv2.bitwise_not(gray_frame)
    _, thresholded_frame = cv2.threshold(inverted_frame, threshold, 255, cv2.THRESH_BINARY)
    
    # Create processed image for trimming.
    processed_frame = inverted_frame #thresholded.copy()
    
#     # Determine center of the frame and offset by prespecified amount.
#     center_x = (frame.shape[1] // 2) + x_offset
#     center_y = (frame.shape[0] // 2) + y_offset
    
#     # Create a mask for the circular region
#     mask = np.zeros_like(processed_frame)
#     cv2.circle(mask, (center_x, center_y), circle_radius, (255, 255, 255), -1)
    
#     # Calculate the coordinates for the square region
#     top_left_x = center_x - square_size // 2
#     top_left_y = center_y - square_size // 2
#     bottom_right_x = center_x + square_size // 2
#     bottom_right_y = center_y + square_size // 2
    
#     # Trim to square region
#     processed_trimmed = processed[top_left_y:bottom_right_y, top_left_x:bottom_right_x]
    
    return processed_trimmed

In [ ]:
# Process video.

# Parameters
threshold_value = 128
square_size = 1360    # Size of the square region to extract
circle_radius = 680   # Radius of circular region to extract

# Open video file
video_path = 'aggregationA.mp4'
output_gif_path = 'aggregationA2.gif'

# Define the start and end times of the segment you want to process (in seconds)
start_time = 1.5 * 60
end_time = 20 * 60

# Load the video clip and extract the desired subclip
video_clip = VideoFileClip(video_path) #.subclip(start_time, end_time)

# Process each frame and trim to square regions
processed_frames = [process_frame(frame, threshold_value, square_size) for frame in video_clip.iter_frames()]

# Close the video clip
video_clip.close()

In [ ]:
# Write the processed frames to a GIF
gif_clip = ImageSequenceClip(processed_frames, fps=video_clip.fps)
gif_clip.write_gif(output_gif_path)

In [97]:
clip = VideoFileClip("aggregationA.mp4")
clip.fps

NameError: name 'VideoFileClip' is not defined

## New Approach to Stitch Together Processed Frames into a Gif

In [13]:
import imageio
import os

# Directory containing the JPEG files
input_dir = "aggregationA_processed_frames_2/"

# Output GIF file path
output_gif_path = "aggregationA2.gif"

# List all JPEG files in the directory
jpg_files = [f for f in os.listdir(input_dir) if f.endswith('.jpg')]

# Sort the files based on their frame number
jpg_files.sort(key=lambda x: int(x.split('_')[1].split('.')[0]))

# Read images and create GIF
images = []
count = 0
factor = 4
for filename in jpg_files:
    if count % factor == 0:
        file_path = os.path.join(input_dir, filename)
        images.append(imageio.imread(file_path))
    count += 1

# Adjust frame duration to control GIF speed (in seconds)
frame_duration = 0.1  # Modify as needed

# Save images as GIF
imageio.mimsave(output_gif_path, images, duration=frame_duration)

print("GIF created successfully!")

GIF created successfully!
